In [1]:
# Libraries
import pandas as pd, numpy as np, seaborn as sn, matplotlib.pyplot as plt, ipywidgets as widgets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from imblearn.ensemble import EasyEnsembleClassifier
from sklearn import metrics, preprocessing

## Fitur Utama Prabu

In [5]:
import joblib
import pandas as pd
import numpy as np
from ipywidgets import IntSlider, FloatSlider, Label, VBox, HBox, Layout
import matplotlib.pyplot as plt

# Load model dari file joblib
best_model = joblib.load("PrabuModule/PrabuModel.joblib")

# Ambil satu sampel data untuk demonstrasi (misalnya baris pertama dari X_test)
if not X_test.empty:
    sample_applicant_data = X_test.iloc[[0]]
    company_name_sample = restricted_df.loc[sample_applicant_data.index, 'Company name'].values[0]
else:
    print("X_test kosong, membuat data aplikan dummy untuk demonstrasi fitur Prabu.")
    if 'X_train' in globals() and not X_train.empty:
        feature_columns = X_train.columns
    else:
        feature_columns = ['MScore.2015.int', 'Turnover.2015', 'EBIT.2015', 'PLTax.2015', 'Leverage.2015', 'ROE.2015', 'TAsset.2015',
                           'MScore.2016.int', 'Turnover.2016', 'EBIT.2016', 'PLTax.2016', 'Leverage.2016', 'ROE.2016', 'TAsset.2016',
                           'MScore.2017.int', 'Turnover.2017', 'EBIT.2017', 'PLTax.2017', 'Leverage.2017', 'ROE.2017', 'TAsset.2017',
                           'MScore.2018.int', 'Turnover.2018', 'EBIT.2018', 'PLTax.2018', 'Leverage.2018', 'ROE.2018', 'TAsset.2018',
                           'Automobiles and Components', 'Capital Goods', 'Commercial and professional services',
                           'Consumer Durables and Apparel', 'Consumer Services', 'Diversified Financials', 'Energy',
                           'Food Beverage and Tobacco', 'Food and Staples Retailing', 'Health Care Equipment and Services',
                           'Household and Personal Products', 'Materials', 'Media & Entertainment',
                           'Pharmaceuticalcs biotechnology and life sci', 'Real Estate', 'Retailing', 'Software and Services',
                           'Technology Hardware and Equipment', 'Telecommunication Services', 'Transportation', 'Utilities',
                           'France', 'Germany', 'Italy', 'Spain']
    dummy_data = np.random.rand(1, len(feature_columns))
    sample_applicant_data = pd.DataFrame(dummy_data, columns=feature_columns)
    company_name_sample = "Perusahaan Dummy"

# 1. Credit Risk Score (0–100)
try:
    proba_default = best_model.predict_proba(sample_applicant_data)[:, 1]
    credit_risk_score = (1 - proba_default[0]) * 100
except Exception as e:
    print(f"Error saat prediksi probabilitas: {e}")
    credit_risk_score = 50.0

score_gauge = FloatSlider(value=credit_risk_score, min=0, max=100, step=0.1, description='Credit Risk Score:', disabled=True, layout=Layout(width='50%'))

# 2. Risk Category
risk_category = ""
category_color = ""
if credit_risk_score >= 70:
    risk_category = "Low Risk"
    category_color = "lightgreen"
elif credit_risk_score >= 30:
    risk_category = "Medium Risk"
    category_color = "yellow"
else:
    risk_category = "High Risk"
    category_color = "salmon"

risk_label = Label(value=f"Risk Category: {risk_category}", layout=Layout(padding='5px', background_color=category_color))

# 3. Score History (per applicant) - Data Dummy
applicant_ids = [company_name_sample, "Perusahaan B", "Perusahaan C"]
score_history_data = {
    company_name_sample: np.random.randint(30, 90, size=12),
    "Perusahaan B": np.random.randint(40, 100, size=12),
    "Perusahaan C": np.random.randint(20, 70, size=12)
}
months = [f"Bulan {i+1}" for i in range(12)]

plt.figure(figsize=(10, 4))
plt.plot(months, score_history_data[company_name_sample], marker='o', label=f"Skor Histori {company_name_sample}")
plt.title(f"Score History untuk {company_name_sample}")
plt.xlabel("Periode")
plt.ylabel("Credit Risk Score")
plt.ylim(0, 100)
plt.xticks(rotation=45)
plt.legend()
plt.grid(True)
plt.tight_layout()
score_history_plot = plt.gcf()
plt.show()

# 4. Approval Recommendation
recommendation = ""
recommendation_icon = ""
if risk_category == "Low Risk":
    recommendation = "Approve"
    recommendation_icon = "✔️"
elif risk_category == "Medium Risk":
    recommendation = "Review"
    recommendation_icon = "❓"
else:
    recommendation = "Decline"
    recommendation_icon = "❌"

recommendation_label = Label(value=f"Rekomendasi: {recommendation_icon} {recommendation}")

display(VBox([score_gauge, risk_label, recommendation_label]))

print(f"\nAnalisis untuk Perusahaan: {company_name_sample}")


NameError: name 'X_test' is not defined

In [ ]:
import json

# Fungsi untuk menghitung M-Score berdasarkan data keuangan
def calculate_m_score(financial_data):
    # Contoh sederhana, sesuaikan dengan logika perhitungan M-Score yang sebenarnya
    # Misalnya, jika laba bersih positif dan ekuitas meningkat, skor lebih rendah (risiko rendah)
    laba_bersih = financial_data.get("Laba tahun berjalan", 0)
    ekuitas = financial_data.get("Jumlah ekuitas", 0)
    
    if laba_bersih > 0 and ekuitas > 100000000000000: # Contoh threshold
        return 'A' # Risiko sangat rendah
    elif laba_bersih > 0:
        return 'B' # Risiko rendah
    elif ekuitas < 50000000000000:
        return 'D' # Risiko sangat tinggi
    else:
        return 'C' # Risiko sedang

try:
    with open('Output/Sarana/hasil_ekstraksi_semua_dokumen.json', 'r') as f:
        sarana_output = json.load(f)
    
    # Asumsi kita menggunakan data dari dokumen pertama di JSON untuk semua perusahaan di clean_df
    # Ini adalah penyederhanaan, idealnya ada pemetaan antara perusahaan di clean_df dan data di JSON
    if sarana_output and len(sarana_output) > 0:
        financial_data_example = sarana_output[0]['hasil_ekstraksi']
        # Buat kolom M-Score baru berdasarkan perhitungan
        # Untuk demonstrasi, kita terapkan M-score yang sama ke semua baris, atau baris tertentu jika ada ID yang cocok
        # Jika Anda memiliki ID perusahaan yang cocok antara clean_df dan sarana_output, gunakan itu untuk pemetaan.
        # Untuk saat ini, kita akan membuat M-score dummy berdasarkan data pertama jika ada.
        
        # Contoh: Ambil beberapa nama perusahaan dari clean_df untuk diberi M-Score dari JSON
        # Ini hanya contoh, idealnya ada mekanisme pencocokan yang lebih baik
        target_companies = clean_df['Company name'].unique()[:len(sarana_output)] # Ambil sebanyak data di JSON
        
        m_scores_from_sarana = {}
        for i, entry in enumerate(sarana_output):
            if i < len(target_companies):
                 # Hitung M-Score dari data keuangan di JSON
                calculated_m_score = calculate_m_score(entry['hasil_ekstraksi'])
                m_scores_from_sarana[target_companies[i]] = calculated_m_score
        
        # Update MScore.2020 di clean_df (atau restricted_df jika sudah dibuat)
        # Kita akan update 'MScore.2020' dan 'MScore.2020.int'
        for index, row in restricted_df.iterrows():
            company_name = row['Company name']
            if company_name in m_scores_from_sarana:
                new_mscore = m_scores_from_sarana[company_name]
                restricted_df.loc[index, 'MScore.2020'] = new_mscore
                restricted_df.loc[index, 'MScore.2020.int'] = 0 if new_mscore.lower() in ['a', 'b'] else 1
        print("Kolom M-Score telah diperbarui menggunakan data dari Output/Sarana.")
    else:
        raise FileNotFoundError # Jika file ada tapi kosong
except FileNotFoundError:
    print("File Output/Sarana/hasil_ekstraksi_semua_dokumen.json tidak ditemukan atau kosong, membuat data M-Score dummy.")
    # Buat data M-Score dummy jika file tidak ada
    # Misalnya, kita buat distribusi acak untuk MScore.2020
    dummy_m_scores = np.random.choice(['A', 'B', 'C', 'CC', 'CCC', 'D'], size=len(restricted_df))
    restricted_df['MScore.2020'] = dummy_m_scores
    restricted_df['MScore.2020.int'] = restricted_df['MScore.2020'].apply(lambda x: 0 if str(x).lower() in ['a', 'b'] else 1)
    print("Kolom M-Score dummy telah dibuat.")

# Pastikan MScore.2019.int juga ada dan benar, karena digunakan untuk balancing
if 'MScore.2019.int' not in restricted_df.columns:
    # Jika tidak ada, buat dummy juga atau gunakan MScore.2020.int sebagai proxy
    print("Kolom MScore.2019.int tidak ditemukan, menggunakan MScore.2020.int sebagai proxy untuk balancing.")
    restricted_df['MScore.2019.int'] = restricted_df['MScore.2020.int']
    
# Re-balancing jika MScore.2019.int baru saja dibuat atau diubah
high_risk_df = restricted_df[restricted_df['MScore.2019.int'] == 1]
low_risk_df = restricted_df[restricted_df['MScore.2019.int'] == 0].sample(n=len(high_risk_df) if len(high_risk_df) > 0 else 0, random_state=0)
if len(high_risk_df) > 0 and len(low_risk_df) > 0:
    restricted_df = pd.concat([low_risk_df, high_risk_df])
    restricted_df.sort_index(inplace=True)
else:
    print("Tidak cukup data untuk balancing setelah update M-Score.")
    # Jika tidak ada cukup data, mungkin kita gunakan seluruh clean_df tanpa balancing MScore.2019.int
    # Atau handle error sesuai kebutuhan
    # Untuk sekarang, kita lanjutkan dengan restricted_df yang mungkin tidak seimbang pada MScore.2019.int
    # Ini bisa mempengaruhi pelatihan model, jadi idealnya ada data yang cukup

restricted_df.head() # Tampilkan hasil untuk verifikasi

# Pastikan kolom yang akan digunakan untuk prediksi (misal MScore.2019.int) ada di restricted_df
# dan variabel target (MScore.2020.int) juga ada.
print("\nDistribusi MScore.2019.int setelah update/dummy:")
print(restricted_df['MScore.2019.int'].value_counts())
print("\nDistribusi MScore.2020.int setelah update/dummy:")
print(restricted_df['MScore.2020.int'].value_counts())

In [ ]:
from sklearn.metrics import  accuracy_score, classification_report

# Function that takes a MScore and cast it into an integer value (0 means a low credit risk, 1 an high credit risk)
def MScore_to_int(x):
    return 0 if (str(x[0]).lower() in ['a', 'b']) else 1

# Function to print the overall performance metrics of a model
def print_performances(name, classifier, X_train, y_train, X_test, y_test):
    y_predict_train = classifier.predict(X_train)
    y_predict_test = classifier.predict(X_test)
    print(name + '\n - Train accuracy: ' + str(round(accuracy_score(y_train, y_predict_train) * 100, 1)) + '%')
    print(' - Test accuracy: '+ str(round(accuracy_score(y_test, y_predict_test) * 100, 1)) + '%')
    print('\nTest '+str(classification_report(y_test, y_predict_test)))